In [1]:
pip install yarl

     |████████████████████████████████| 271 kB 10.2 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 


In [47]:

import yarl
from yarl import URL
url = URL("https://github.com/search?q=data+science")

In [48]:
url.scheme

'https'

In [49]:
url.path

'/search'

In [50]:
url.query_string

'q=data science'

In [51]:

url = URL("https://github.com/khuyentran1401/Data-science#contents")

In [52]:
url.fragment

'contents'

In [53]:
import pandas as pd
import os
data = pd.read_csv("URL Classification.csv", names=["url", "Type"], index_col=0)

In [54]:
data

,url,Type
1,http://www.liquidgeneration.com/,Adult
2,http://www.onlineanime.org/,Adult
3,http://www.ceres.dti.ne.jp/~nekoi/senno/senfir...,Adult
4,http://www.galeon.com/kmh/,Adult
5,http://www.fanworkrecs.com/,Adult
...,...,...
1562974,http://www.maxpreps.com/,Sports
1562975,http://www.myscore.com/,Sports
1562976,http://sportsillustrated.cnn.com/highschool,Sports
1562977,http://rss.cnn.com/rss/si_highschool?format=xml,Sports


In [55]:
sample = data.sample(10000, random_state=1)
sample

,url,Type
1308349,http://yorkrite.com/ne/gcram/,Society
276825,http://www.alliedartistsofamerica.org/,Arts
1315267,http://www.msstate.edu/org/farmhouse/index.html,Society
392387,http://www.sunnytec.com.tw/,Business
1267730,http://www.kabissa.org,Society
...,...,...
169463,http://www.naxos.com/composerinfo/3303.htm,Arts
1094461,http://members.tripod.com/arroweb1/,Science
526843,http://www.petroskills.com/,Business
171473,http://www.gregbartholomew.com,Arts


In [56]:
sample["url"] = sample["url"].apply(lambda url: URL(url))

In [57]:
sample


,url,Type
1308349,http://yorkrite.com/ne/gcram/,Society
276825,http://www.alliedartistsofamerica.org/,Arts
1315267,http://www.msstate.edu/org/farmhouse/index.html,Society
392387,http://www.sunnytec.com.tw/,Business
1267730,http://www.kabissa.org,Society
...,...,...
169463,http://www.naxos.com/composerinfo/3303.htm,Arts
1094461,http://members.tripod.com/arroweb1/,Science
526843,http://www.petroskills.com/,Business
171473,http://www.gregbartholomew.com,Arts


In [58]:

sample["url"] = sample["url"].apply(lambda url: URL(url))

processed = sample.assign(
    host=sample.url.apply(lambda url: url.host),
    path=sample.url.apply(lambda url: url.path),
    name=sample.url.apply(lambda url: url.name),
    scheme=sample.url.apply(lambda url: url.scheme),
    query=sample.url.apply(lambda url: url.query_string),
)
processed

,url,Type,host,path,name,scheme,query
1308349,http://yorkrite.com/ne/gcram/,Society,yorkrite.com,/ne/gcram/,,http,
276825,http://www.alliedartistsofamerica.org/,Arts,www.alliedartistsofamerica.org,/,,http,
1315267,http://www.msstate.edu/org/farmhouse/index.html,Society,www.msstate.edu,/org/farmhouse/index.html,index.html,http,
392387,http://www.sunnytec.com.tw/,Business,www.sunnytec.com.tw,/,,http,
1267730,http://www.kabissa.org,Society,www.kabissa.org,/,,http,
...,...,...,...,...,...,...,...
169463,http://www.naxos.com/composerinfo/3303.htm,Arts,www.naxos.com,/composerinfo/3303.htm,3303.htm,http,
1094461,http://members.tripod.com/arroweb1/,Science,members.tripod.com,/arroweb1/,,http,
526843,http://www.petroskills.com/,Business,www.petroskills.com,/,,http,
171473,http://www.gregbartholomew.com,Arts,www.gregbartholomew.com,/,,http,


In [59]:
# Group by the columns Type and host and get the count
group = processed.groupby(["Type", "host"]).agg(count=("url", "count"))

# Get the top 5 most popular hosts for each type
sorted_group = group.sort_values(by="count", ascending=False).reset_index()
largest = sorted_group.groupby("Type").head(5).sort_values(by='Type')

# View the bottom 10 rows
largest.tail(10)

,Type,host,count
17,Society,members.tripod.com,12
25,Society,en.wikipedia.org,9
7,Society,www.angelfire.com,25
3,Society,www.geocities.com,56
1,Society,www.newadvent.org,91
35,Sports,sportsillustrated.cnn.com,7
11,Sports,www.geocities.com,18
45,Sports,www.clubwebsite.co.uk,6
50,Sports,www.angelfire.com,6
47,Sports,www.freewebs.com,6


In [21]:
pip install graphistry

     |████████████████████████████████| 127 kB 9.9 MB/s 


In [65]:
! pip install --user graphistry
import graphistry
print(graphistry.__version__)

graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="whitecaspian", password="*********") 

0.23.2


In [66]:
edges = largest[["Type", "host"]]

In [67]:
def create_node_df(df: pd.DataFrame, col_name: str):
  """Create a node DataFrame for a specific column"""
  nodes = ( df[[col_name]].assign(type=col_name).rename(columns={col_name: "node"}).drop_duplicates())
  return nodes
    


In [68]:
    
type_nodes = create_node_df(largest, "Type")
url_nodes = create_node_df(largest, "host")
nodes = pd.concat([type_nodes, url_nodes])
nodes

,node,type
115,Adult,Type
12,Arts,Type
114,Business,Type
9,Computers,Type
41,Games,Type
127,Health,Type
18,Home,Type
111,Kids,Type
1343,News,Type
4,Recreation,Type


In [69]:
g = (
    graphistry
    .edges(edges, "Type", "host")
    .nodes(nodes, "node")
)

g.plot()

In [73]:
!pip install faker

     |████████████████████████████████| 1.5 MB 8.6 MB/s 


In [74]:

from faker import Faker

types = list(edges.Type.unique())

fake = Faker()
colors = [fake.color() for _ in range(len(types))]
node_color_mapping = dict(zip(types, colors))
node_color_mapping

{'Adult': '#0dc416',
 'Arts': '#49d644',
 'Business': '#f24fea',
 'Computers': '#a8abea',
 'Games': '#e3ed2d',
 'Health': '#e85ab4',
 'Home': '#eaccff',
 'Kids': '#9ff4ef',
 'News': '#85e2a8',
 'Recreation': '#261c8e',
 'Reference': '#f9b486',
 'Science': '#e6fc79',
 'Shopping': '#f22bc7',
 'Society': '#d442f4',
 'Sports': '#e86a30'}

In [75]:
node_icon_mapping = {"host": "link", "Type": "newspaper-o"}

In [76]:

g = (
    graphistry.edges(edges, "Type", "host")
    .nodes(nodes, "node")
    .encode_point_color(
        "node", categorical_mapping=node_color_mapping, default_mapping="silver"
    )
    .encode_point_icon(
        "type", categorical_mapping=node_icon_mapping
    )
)

g.plot()